# XHARPy Example: Urea with Quantum Espresso
 - Program: Quantum Espresso
 - B3LYP calculation from PBE files
 - Core scaling

In [ ]:
from xharpy import (shelxl_hkl2pd, cif2data, lst2constraint_dict, create_construction_instructions,
                    refine, write_cif, write_res, write_fcf, add_density_entries_from_fcf)
import os

In [ ]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

### Loading Data

In [ ]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)

In [ ]:
hkl = shelxl_hkl2pd('iam.hkl')

In [ ]:
constraint_dict = lst2constraint_dict('iam.lst')

### Setting Options

In [ ]:
refinement_dict = {
    'f0j_source': 'qe',
    'reload_step': 1,
    'core': 'scale',
}

In [ ]:
computation_dict = {
    'symm_equiv': 'once',
    'mpicores': 2,
    'control': {
        'prefix': 'A23',
        'pseudo_dir': './pseudo/',
    },
    'system': {
        'ibrav': 6,
        'a': float(cell[0]),
        'c': float(cell[2]),
        'ecutwfc': 50,
        'ecutrho': 200,
        'input_dft': 'b3lyp'
    },
    'paw_files': {
        'H': 'H.pbe-kjpaw_psl.1.0.0.UPF',
        'C': 'C.pbe-n-kjpaw_psl.1.0.0.UPF',
        'O': 'O.pbe-n-kjpaw_psl.1.0.0.UPF',
        'N': 'N.pbe-n-kjpaw_psl.1.0.0.UPF'
    },
    'k_points':{
        'mode': 'automatic',
        'input': '1 1 1 0 0 0'
    }
}

### Refinement

In [ ]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [ ]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-10 07:24:38.275070
Preparing


Traceback (most recent call last):
  File "core.py", line 67, in <module>
    f0j[start_index:end_index] = f_core_from_spline(nc, g_ks[start_index:end_index] * Bohr, k=19)
  File "core.py", line 18, in f_core_from_spline
    j0[gr != 0] = np.sin(2 * np.pi * gr[gr != 0]) / (2 * np.pi * gr[gr != 0])
KeyboardInterrupt
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



  calculating the core structure factor for O
Traceback (most recent call last):
  File "/home/niklas/anaconda3/envs/xharpy/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_5496/1164851934.py", line 1, in <module>
    parameters, var_cov_mat, information = refine(
  File "/home/niklas/Documents/har/xharpylib/xharpy/core.py", line 1458, in refine
    f0j_core = jnp.array(calc_f0j_core(cell_mat_m, type_symbols, constructed_xyz, index_vec_h, symm_mats_vecs, computation_dict))
  File "/home/niklas/Documents/har/xharpylib/xharpy/f0j_sources/gpaw_mpi_source.py", line 650, in calc_f0j_core
    res = subprocess.run('python core.py', shell=True)
  File "/home/niklas/anaconda3/envs/xharpy/lib/python3.8/subprocess.py", line 495, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/home/niklas/anaconda3/envs/xharpy/lib/python3.8/subprocess.py", line 1020, in

TypeError: object of type 'NoneType' has no len()

### Writing Data to Disk

In [ ]:
write_cif(
    output_cif_path=os.path.join(output_folder, 'xharpy.cif'),
    cif_dataset='xharpy',
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_res(
    out_res_path=os.path.join(output_folder, 'xharpy_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [ ]:
add_density_entries_from_fcf(
    os.path.join(output_folder, 'xharpy.cif'),
    os.path.join(output_folder, 'xharpy_6.fcf')
)